# Composite AutoML

In [1]:
import pandas as pd

df = pd.read_csv('tmp/extracted_data_cleaned_final.csv')
X = df.iloc[:,:-1]
X = X.loc[:, ~X.columns.str.contains('^Unnamed')]
y = df.iloc[:,-1]

df.head()

,Unnamed: 0,"Zone17_Pressure__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.0","Zone9_Pressure__fft_coefficient__coeff_48__attr_""imag""","Zone17_Pressure__fft_coefficient__coeff_34__attr_""abs""","Zone17_Pressure__fft_coefficient__coeff_74__attr_""abs""","Zone28_Pressure__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.6",Zone17_Pressure__spkt_welch_density__coeff_5,Zone16_Pressure__count_below_mean,"Zone17_Pressure__fft_coefficient__coeff_33__attr_""abs""","Zone10_Pressure__fft_coefficient__coeff_32__attr_""imag""",...,"Zone17_Pressure__fft_coefficient__coeff_98__attr_""real""","Zone15_Pressure__augmented_dickey_fuller__attr_""pvalue""","Zone16_Pressure__fft_aggregated__aggtype_""kurtosis""","Zone17_Pressure__fft_coefficient__coeff_47__attr_""abs""","Zone17_Pressure__fft_coefficient__coeff_24__attr_""abs""","Zone15_Pressure__augmented_dickey_fuller__attr_""teststat""","Zone11_Pressure__fft_coefficient__coeff_32__attr_""abs""","Zone9_Pressure__fft_coefficient__coeff_15__attr_""abs""","Zone15_Pressure__fft_coefficient__coeff_50__attr_""angle""",target
0,0,78.888668,203207.271665,335998.964510,174790.648082,740.268452,4.258907e+05,711.0,317571.085145,204831.749088,...,-40686.491363,0.581636,-6.653085,184254.593503,366290.844718,-1.401334,463314.004631,761641.585997,92.291990,1
1,1,81.348721,252456.190656,264661.383781,166894.957965,750.602367,4.900410e+05,723.0,314088.033794,487031.962977,...,-70076.943493,0.705610,-6.678313,119136.759897,138138.585650,-1.123499,853980.652251,860928.464443,-8.645479,0
2,2,80.815937,91712.328390,353344.385327,141080.311072,647.083908,2.445339e+05,752.0,423629.104033,30447.202237,...,-52448.149267,0.475156,-6.362571,263383.759531,581463.747023,-1.615374,195138.646235,301922.912358,72.981350,1
3,3,80.860089,251598.722519,279051.936161,127273.704379,605.459887,2.224586e+07,718.0,269752.828528,472488.210871,...,-56904.774944,0.393912,-7.101364,238767.019112,534992.124308,-1.773068,768603.411764,684944.504578,42.942366,0
4,4,85.236354,125865.057396,206649.731446,197217.256490,174.224324,2.360030e+07,733.0,230455.169767,356545.758714,...,-92365.101674,0.246432,-6.859449,231696.020125,465391.833486,-2.095243,583046.908395,316534.010743,-100.266190,0


### TPOT

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25)

# generations: int, optional (default=100)
#    Number of iterations to the run pipeline optimization process. Must be a positive number.
#    Generally, TPOT will work better when you give it more generations (and therefore time) to optimize the pipeline.
    TPOT will evaluate population_size + generations × offspring_size pipelines in total. 
tpot = TPOTClassifier(generations=100, population_size=50, verbosity=2, n_jobs=-1)
tpot.fit(X_train, y_train)

print(tpot.score(X_test, y_test))